# Decoding color codes by turning all lights off

Here, we summarize a main use case of the color-code decoder package of the MQT-QECC tool.

## Quantum Codes
As in classical computing, a quantum code protects information from errors by adding redundancy in a certain way to make the information less fragile. A classical binary code is defined as a vector space s.t. each vector in that space fulfills a set of constraints, called `checks`. For binary codes, the checks of the code are parity checks that determine whether the sum (mod 2) of some entries of a vector is odd or even (equal to 1 or 0). 
An important class of quantum codes, `CSS` codes can be seen as a certain combination of two classical codes that is, they have two sets of checks. We consider a simple bit-flip noise model in the following, which implies that we can treat both sets of checks equivalently, so we will focus on only a single one for the remainder of this notebook.

An example of a quantum code is presented in the following figure.


<img src="images/cc.png" alt="Triangular color code" width="50%">

This is a quantum color code, that is defined on a three-valent and three-colorable lattice with triangular shape. 
Each vertex corresponds to a single bit of the code and the checks correspond to the faces, i.e., each face computes the sum (mod 2) of the bits around it. If a vector satisfies all checks, it is a codeword of the code. However, in case an error occurs on a single bit, the checks in which the bit is involved in, i.e., the faces around the vertex of the bit will indicate that the sum is not even anymore, and hence will indicate an error happened. This is illustrated in the following figure for a small triangular color code
<img src="images/steane-ex.png" alt="Color code example" width="15%">

The decoding task is now, given a set of violated checks, i.e., marked faces, to find a set of bits to flip s.t. no check is violated anymore. In the case of the example above, flipping the bit on the very top of the triangle is a decoding estimate.

## LightsOut puzzle

We solve the problem of decoding quantum codes by a reduction to the well-known mathematical puzzle `LightsOut`. Originally, the puzzle is defined on a rectangular lattice whose faces correspond to lights that can be turned on or off. Pressing a light toggles the light (from on to off and vice versa) all adjacent lights around it as well. The puzzle starts out in an initial configuration where some lights are turned on and the goal is to find a sequence of lights to press (a seqeuence of `moves` s.t. all lights are turned off), as illustrated in the following figure:

<img src="images/lo.png" alt="LightsOut puzzle" width="50%">

To show how this can be used to decode a code, consider the following variation where: the faces correspond to lights and the vertices correspond to switches that toggle all lights around them. For the triangular color code, the LightsOut variant and an example of an initial configuration and a solution to the puzzle is given in the following figure

<img src="images/lights-out.png" alt="LightsOut color code" width="50%">

Our decoder uses a satisfiability formulation (MaxSAT) of the LightsOut puzzle, to find a minimal set of switches that turns off all the lights. 


## Using QECC to decode quantum codes using the LightsOut solution

First, we need to construct a lattice that corresponds to the lattice of the LightsOut puzzle (i.e., the underlying quantum code). For this we use the `HexagonalColorCode` object which constructs a hexagonal lattice with triangular shape as depicted above, given the side length as only parameter.



In [ ]:
from mqt.qecc.cc_decoder.hexagonal_color_code import HexagonalColorCode

side_length = 3
lattice = HexagonalColorCode(3)

Then, we using the lattice, we construct a LightsOut puzzle object that consists of adjacency lists of the lattice, as well as the corresponding satisfiablity formulation. In the formulation, each switch and each light corresponds to a Boolean propositional variable and each check is realized as a parity constraint. If no light is turned on initially, all checks have to be equal to 0.

In [ ]:
from mqt.qecc.cc_decoder.decoder import LightsOut

problem = LightsOut(lattice.faces_to_qubits, lattice.qubits_to_faces)

Now we can choose an initial configuration for the puzzle by defining a list that indicates which lights are initially on. Let's say the initial lights are stored in a list `inital_lights`.

In [ ]:
import numpy as np

initial_lights = np.zeros(7)
initial_lights[0] = 1
lights = [bool(b == 1) for b in intial_lights]

Now we pass this list, together with the MaxSAT solver we want to use (Z3 per default) to the `solve` method, which will set the value of the parity constriants of the faces in the initial configuration accordingly and then call the solver to find a minimal solution set. 

In [ ]:
switches, constr_time, solve_time = problem.solve(lights, solver_path=solver_path)

the `solve` method returns a set of switches that constitute a solution to the LightsOut puzzle, together with the total time needed to construct the satisfiability formulation and the solving time of the MaxSAT sovler.

Check out the [reference documentation](https://qecc.readthedocs.io/en/latest) for more information.
